# Dispersal in metacommunities
![basic dispersal](../resources/basic_dispersal.png)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import ipywidgets as widgets
%matplotlib widget

## Vellend's dispersal

Vellend's basic description is incredibly simple. Say we have some number of patches $M$ making up our metacommunity, each with the same number of individuals. The only change we have to make is that with probability $m$ we choose a reproducing individual from the entire metacommunity rather than only the local community.

$$
P_{migration} = m
$$

Conceptually, and in the code, this mean that now we also have to check each patch in addition to each individual in each year. Now we have a total number of timesteps $=Y \times N \times M$. 

One other change is that when we choose an individual to die we also have to chose which patch the death occurs in. We uniformly select an individual in the entire metacommunity at each time step.

Other than that the basic idea of the moran model changes very little.

In [2]:
def moran_dispersal(n_indiv, n_years, m_patches, init_f1, dispersal):
    """ Implements the Moran model including constant selection and simulates it in time. 
        
        Parameters
        ----------
        n_indiv: int
            number of individuals in the community
        n_years: int
            number of potential 'turnovers' of the community
        m_patches: int
            number of separate patches linked by dispersal
        init_f1: float
            initial frequency of species 1
        dispersal: float
            probability that a new birth is drawn from metacommunity
            
        Return
        ------
        moran: np.array (n_years, 2)
            contains the species frequencies for each year of the simulation """

    # For this we're going to track only species 1 and instead of the columns representing
    # different species they will be different patches
    moran = np.zeros((n_years, m_patches))
    moran[0] = init_f1

    # Now our community vector is instead a metacommunity matrix
    meta = np.random.choice([0, 1], size=(m_patches, n_indiv), replace=True, p=[init_f1, 1 - init_f1])

    # now we can do the loop as in vellend. I'm going to write it a bit differently
    # but the idea is basically the same. (yes its slower)
    for year in tqdm(range(1, n_years)):

        # we now need to check each patch as well as each year
        for patch in range(m_patches):

            # for each year in each patch we do replacement for every individual
            for indiv in range(n_indiv):

                # select patch where death occurs
                death_patch = np.random.choice(range(m_patches))

                # now every time we replace an individual we need to check whether or not
                # dispersal occurs. Draw from entire metacommunity
                if np.random.uniform() < dispersal:
                    pr1 = np.sum(meta == 0) / (n_indiv * m_patches)
                # or draw from local community
                else:    
                    pr1 = np.sum(meta[death_patch] == 0) / n_indiv

                # replace one individual with another
                death = np.random.randint(n_indiv)
                birth = np.random.choice([0, 1], p=[pr1, 1 - pr1])
                meta[death_patch, death] = birth

            # when we're done looping over all of the individuals we can update
            # frequencies for the year (the timescale we care about)
            f1 = np.sum(meta[patch] == 0) / n_indiv
            moran[year, patch] = f1

    return moran

In [3]:
def draw_simulation(individuals=250, years=50, patches=10, initial_freq=0.5, dispersal=0.0):
    # the plot bit, this just makes a blank figure
    fig, ax = plt.subplots(ncols=2, figsize=(10,4), gridspec_kw = {'width_ratios':[3, 1]})

    # trajectory labels
    ax[0].set_xlabel('Years')
    ax[0].set_ylabel('Species 1 frequency by patch')
    ax[0].set_ylim([0, 1])
    # distribution labels and bins
    hist_bins = np.arange(0, 1.1, 0.1)
    ax[1].set_xlabel('Count')
    ax[1].set_xlim([0, patches])
    ax[1].set_ylim([0, 1])

    # we're going to plot a distribution too need an array for it
    dist = np.zeros(patches)

    simulation = moran_dispersal(individuals, years, patches, initial_freq, dispersal)
    ax[0].plot(range(years), simulation)
    # add final freq to our dist array
    dist = simulation[-1, :]

    # plot the distribution too
    ax[1].hist(dist, bins=hist_bins,  orientation='horizontal')
    ax[1].axhline(np.mean(dist), linestyle='--')


    plt.tight_layout()    
    plt.show()

# set up the interface
widgets.interact_manual(draw_simulation, individuals=(10, 1000, 10), years=(5, 200, 5),
                  patches=(1, 20, 1), initial_freq=(0.0, 1.0, 0.01), dispersal=(0, 1.0, 0.01))

interactive(children=(IntSlider(value=250, description=&#39;individuals&#39;, max=1000, min=10, step=10), IntSlider(va…

&lt;function __main__.draw_simulation(individuals=250, years=50, patches=10, initial_freq=0.5, dispersal=0.0)&gt;